In [2]:
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
import pandas as pd
import numpy as np 
import torch
from torch import nn
import torch.utils.data as data
import torch.nn.functional as F
from torch import tensor

In [10]:

#先把csv中一个句子拿出来  然后把每个单词去词训练库里面找对应的向量规定对于每个句子矩阵都构建一个2000,300的矩阵。
def hang_vectoes_label(index): # 要读取csv文本第几行的文章数据 会返回一个 列表里面是文章的词向量矩阵和标签值
    row=df.values[index] #读取数据的第几行
    row=row[0].split()   #这些数据变成一个列表
    label=row[0]
    hang_txt=row[1:]
    hang_labeltxt_juzhen=[]
    meihang_wenben_juzhen=np.zeros((2000,300))#，每一个文本定义一个空的全0的2000,300的矩阵 
    t=0
    for a in hang_txt:
        try:
            vec = new_model[a]  #从词库里面取出这些词对应的向量 
        except KeyError:        #当在词向量模型中没有这个单词的向量的时候可以设置错误跳过 然后把这个单词的向量设置为全0
            vec=np.zeros((1,300))
        meihang_wenben_juzhen[int(t)]=vec
        if t ==1999:
            break
        t=t+1
    hang_labeltxt_juzhen.append(label)
    hang_labeltxt_juzhen.append(meihang_wenben_juzhen)
    return hang_labeltxt_juzhen
# a=hang_vectoes_label(0)  #返回一个列表 第一个是行文本的分类的label 第二个是行文本 2000*300 的词向量矩阵 
# # 这个原始数据有1730条文本 数据 



#手动对数据分块 并且把数据加载到dataloader 让dataloader分batch
def dataloader_batch(start,end,batch,shuffle):# 把多少行的数据加入到这个data.TensorDataset中 然后在对这个中的这些行数据分batch训练
            #     开始     终止 batch大小  是否打乱 True False 
    x=[]
    y=[]
    for i in range(int(start),int(end),1):
        a=hang_vectoes_label(i)
        x_1=a[1]
        y_1=int(a[0])
        x.append(x_1)
        y.append(y_1)
    x=torch.Tensor(x)
    y=torch.Tensor(y)
    torch_dataset = data.TensorDataset(x, y) #要求这两部分 第一个维度是相同的 
    loader = data.DataLoader(
        dataset=torch_dataset,
        batch_size=batch,             # 每批提取的数量
        shuffle=shuffle,             # 要不要打乱数据（打乱比较好）
        num_workers=0)            # 多少线程来读取数据
    return loader 



#定义模型优化器
class BiRNN(nn.Module):
    def __init__(self):
        super(BiRNN, self).__init__()
        
        self.embed_size=300  #样本得输入维度 
        self.num_hiddens=100 # 隐藏层神经元的个数
        self.num_layers=2
        self.encoder = nn.LSTM(input_size=self.embed_size, 
                                hidden_size=self.num_hiddens, 
                                num_layers=self.num_layers,
                                bidirectional=True)
        
        self.decoder = nn.Linear(4*self.num_hiddens, 14)

    def forward(self, x):
        x=x.reshape(2000,16,300)
        #                batch大小
#         print(x.shape)
        x=torch.Tensor(x)
        outputs, _ = self.encoder(x) 
        encoding = torch.cat((outputs[0], outputs[-1]), -1)
        outs = self.decoder(encoding)
        outs = F.sigmoid(outs)
        return outs

model = BiRNN()
# model.aux_logits=False
criterion = torch.nn.CrossEntropyLoss()  # 损失函数的计算 交叉熵损失函数计算
# optimizer=optim.SGD(model.parameters(),lr=0.01,momentum=0.5)
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)  # 优化器
# momentum是优化梯度的方向自动适应学习率


In [6]:
# glove2word2vec("text_train_glove_vectors.txt", "text_train_vectors_wd2.txt") #
#把glove训练好的文件转成Wordvec可以读的 如果转过了这个就注释掉 
new_model = KeyedVectors.load_word2vec_format('text_train_vectors_wd2.txt', binary=False)

df=pd.read_csv("train_set.csv")
# with open("train_set.csv", 'r') as f: #计算长度
#     hang_count=len(f.readlines())

In [11]:

for i in range(1): #全部是6
    print("第"+str(i)+"个16条数据内")
    loader=dataloader_batch(i*16,i*16+16,16,True) 
    if __name__ == '__main__':
        for epoch in range(100):    # 对整套数据训练3次
             for step, (batch_x, batch_y) in enumerate(loader):  # 每一步loader释放一小批数据用来学习
                    input=batch_x
                    target=batch_y
                    outputs=model(input)#经过这个模型输出的是 如果批大小是4 就输出四行 每行是14个分类的 二维tensor ,
                    target =target.long() #转化成长整形张量进行计算 loss
                    loss = criterion(outputs, target)
                    loss.backward()
                    optimizer.step()
                    print("第"+str(i)+"个320条数据内","epoch世代:", epoch, "第"+str(step)+"个batch:","这个batch的损失值" ,loss.item())                    
#保存训练模型
torch.save(model.state_dict(), 'trainModel.pth')
#模型加载：
# model = BiRNN()
# model.load_state_dict(torch.load('trainModel.pth'))
# model.eval()    

第0个16条数据内
第0个320条数据内 epoch世代: 0 第0个batch: 这个batch的损失值 2.638547658920288
第0个320条数据内 epoch世代: 1 第0个batch: 这个batch的损失值 2.6371188163757324
第0个320条数据内 epoch世代: 2 第0个batch: 这个batch的损失值 2.6349856853485107
第0个320条数据内 epoch世代: 3 第0个batch: 这个batch的损失值 2.6338651180267334
第0个320条数据内 epoch世代: 4 第0个batch: 这个batch的损失值 2.6308698654174805
第0个320条数据内 epoch世代: 5 第0个batch: 这个batch的损失值 2.626453399658203
第0个320条数据内 epoch世代: 6 第0个batch: 这个batch的损失值 2.6231188774108887
第0个320条数据内 epoch世代: 7 第0个batch: 这个batch的损失值 2.6184072494506836
第0个320条数据内 epoch世代: 8 第0个batch: 这个batch的损失值 2.6135809421539307
第0个320条数据内 epoch世代: 9 第0个batch: 这个batch的损失值 2.6054399013519287
第0个320条数据内 epoch世代: 10 第0个batch: 这个batch的损失值 2.5990264415740967
第0个320条数据内 epoch世代: 11 第0个batch: 这个batch的损失值 2.590970754623413
第0个320条数据内 epoch世代: 12 第0个batch: 这个batch的损失值 2.58365535736084
第0个320条数据内 epoch世代: 13 第0个batch: 这个batch的损失值 2.576279878616333
第0个320条数据内 epoch世代: 14 第0个batch: 这个batch的损失值 2.5667366981506348
第0个320条数据内 epoch世代: 15 第0个batch: 这个batch的损失值 2